In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [5]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI , OpenAIEmbeddings

# Load Documents

loader = WebBaseLoader(
    web_path=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)

docs = loader.load()


# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# MMR 알고리즘 (Maximal Magginal Relevance)
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_core.prompts import ChatPromptTemplate

# RAG-Fusion: 관련 검색 쿼리 생성

# template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
# Generate multiple search queries related to: {question} \n
# Output (4 queries):"""


template = """당신은 주어진 하나의 질문을 기반으로 여러 검색 쿼리를 생성하는 유용한 조수입니다. \n
다음 질문과 관련된 여러 검색 쿼리를 생성하세요: {question} \n
출력 (4개의 쿼리):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


## RRF
- RRF 점수 = 1 / (  K + 순위)
- k 는 정해진 상수로, 일반적으로 60 으로 할당
- 순위가 낮을수록 높은 점수를 부여해서, 여러 순위를 종합한 총점을 매겨서 최종 순위를 조정


In [8]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """
    여러개의 순위가 매겨진 문서 리스트를 받아, RRF 공식을 사용하여 문서의 최종 순위를 계산하는 함수.
    k 는 RRF 공식에서 사용되는 선택적 파라미터
    top_n  은 반환할 우선순위가 높은 문서의 개수
    """

    # 각 고유한 문서에 대한 점수를 저장할 딕셔너리를 초기화
    fused_score = {}

    # 순위가 매겨진 문서 리스트를 순회합니다.
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)

            if doc_str not in fused_score:
                fused_score[doc_str] = 0
            previous_score = fused_score[doc_str]

            fused_score[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_score.items(), key=lambda x:x[1],
        reverse=True)
    ]

    return reranked_results[:top_n]

In [9]:
retriever_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

# 체인을 실행
question = "향후 집값에 대해서 알려줘"
docs = retriever_chain_rag_fusion.invoke({"question": question})

len(docs)

docs


/var/folders/2j/vyd5f6nn00b37z36r3jmx9fh0000gn/T/ipykernel_98853/1852983741.py:25: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


[(Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='서울 아파트 전세가율 50.92% 2025년 매매 11.26%↑ 전세 3.83%↑ 서초·송파 등 9개구 역대 최저 서울 주요 지역의 아파트 전세가율(매매가 대비 전세가 비율)이 역대 최저로 떨어졌다. 전세가격이 올랐\n\n\n세계일보\n\n51분전'),
  0.06479774705581157),
 (Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='◀ 앵커 ▶ 시장 반응은 어떨까요? 집을 내놓을까 고민하는 사람들도 있지만, 버티면 결국 이긴다는 분위기가 여전했습니다. 부동산 시장 맹신을 꺾으려면 어떻게 해야 할까요? 기자의 눈, 이준희 기자입니다. ◀ 리포트 \n\n\nMBC\n\n3시간전'),
  0.04972677595628415)]

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """ 다음 맥락을 바탕으로 질문에 답변하세요
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retriever_chain_rag_fusion     ,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

'현재 서울 아파트 전세가율이 50.92%로 역대 최저를 기록하고 있으며, 2025년에는 매매가가 11.26% 상승하고 전세가는 3.83% 상승할 것으로 예상되고 있습니다. 이러한 데이터는 향후 집값이 상승할 가능성이 있음을 시사합니다. 그러나 시장에서는 집을 내놓을까 고민하는 사람들도 있지만, 많은 이들이 버티면 결국 이길 것이라는 분위기를 유지하고 있습니다. 따라서 향후 집값은 상승세를 보일 가능성이 있지만, 시장의 반응과 개인의 선택에 따라 다를 수 있습니다.'